In [10]:
import requests
import pandas as pd

In [11]:
# ref : https://github.com/infoBMKG/data-gempabumi

autogempa = "https://data.bmkg.go.id/DataMKG/TEWS/autogempa.json"
gempabumiterkini = "https://data.bmkg.go.id/DataMKG/TEWS/gempaterkini.json"
gempabumidirasakan = "https://data.bmkg.go.id/DataMKG/TEWS/gempadirasakan.json"

In [132]:
class DataBMKG():
    def __init__(self, data: str):
        self.__get_bmkg(data)

    def __auto_earthquake(self, filename, data: dict):
        data = pd.DataFrame.from_dict(data, orient='index').T
        data.to_csv('dataset/'+filename+'.csv', index=True)

    # need to improve table for more tidy csv from felt to new
    def __felt_earthquake(self, filename, data: pd.DataFrame):
        columns = ['Tanggal','Jam','DateTime','Coordinates','Lintang','Bujur','Magnitude','Kedalaman','Wilayah', 'Dirasakan']

        rows = []
        for d in data:
            rows.append(pd.Series(d, index=columns))
        
        datas = pd.DataFrame(rows, columns=columns)
        datas.to_csv('dataset/'+filename+'.csv', index=True)
            
    def __new_earthquake(self, filename, data: pd.DataFrame):
        columns = ['Tanggal','Jam','DateTime','Coordinates','Lintang','Bujur','Magnitude','Kedalaman','Wilayah', 'Potensi']

        rows = []
        for d in data:
            rows.append(pd.Series(d, index=columns))
        
        datas = pd.DataFrame(rows, columns=columns)
        datas.to_csv('dataset/'+filename+'.csv', index=True)

    def __get_bmkg(self, data:str):

        from io import StringIO
        
        try:
            headers = {
                'User-Agent': 'Mozilla/5.0 \
                    (Macintosh; Intel Mac OS X 10_11_5) \
                    AppleWebKit/537.36 (KHTML, like Gecko) \
                    Chrome/50.0.2661.102 Safari/537.36'
            }
            
            response = requests.get(data, headers=headers)
            raw = pd.read_json(StringIO(response.content.decode('utf-8')))
            
            filename = data.split('/')[-1].split('.')[0]

            if filename == 'autogempa':
                self.__auto_earthquake(filename, raw['Infogempa']['gempa'])
            elif filename == 'gempadirasakan':
                self.__felt_earthquake(filename, raw['Infogempa']['gempa'])
            elif filename == 'gempaterkini':
                self.__new_earthquake(filename, raw['Infogempa']['gempa'])
                
        except requests.exceptions.HTTPError as errh:
            print(repr(errh))
        except requests.exceptions.ConnectionError as errc:
            print(repr(errc))
        except requests.exceptions.Timeout as errt:
            print(repr(errt))
        except requests.exceptions.RequestException as err:
            print(repr(err))

# Data BMKG Gempa

In [133]:
DataBMKG(autogempa)

# Data BMKG Gempa Dirasakan

In [134]:
DataBMKG(gempabumidirasakan)

# Data BMKG Gempa Terkini

In [135]:
DataBMKG(gempabumiterkini)